#Uvod u PyDistSim

Cilj ove vježbe je napraviti pregled najbitnijih komponenti paketa PyDistSim za simulaciju raspodijeljenih algoritama.

## 'Hello distributed world'

U ovom primjeru analiziramo implementaciju jednog od najjednostavnijih algoritma koji riješava problem *Broadcast*: `Flood`. Cilj algoritma je podijeliti informaciju koju posjeduje jedan čvor ostalim čvorovima u mreži.

U ovom primjeru koristi se interkativna konzola IPython te se stoga preporuča najprije napraviti import najvažnijih klasa i funkcija paketa pydistsim na slijedeći način:

In [1]:
from pydistsim import *

ModuleNotFoundError: No module named 'PySide'

Ukoliko se konzola pokreće programom `ipydistsim`, u kojem je preddefinirano korištenje profila pydistsim tada je ovaj import već učinjen prilikom starta.

### Kreiranje mreže

Mreža kao osnovna struktura može se kreirati na više načina. U većini slučajeva sasvim dovoljno je koristiti klasu `NetworkGenerator` kojoj se prilikom instanciranja pridruže željeni parametri kao što su broj čvorova (točan, maksimalan, minimalan) prosječan broj susjeda itd...

U ovom primjeru jedini parametar koji se želi imati pod kontrolom je broj čvorova, a iznosi 100:

In [ ]:
net_gen = NetworkGenerator(100)

Nakon toga možemo koristiti metode generatora kako bi izradili nove mreže s željenim parametrima, primjerice za nasumičan raspored čvorova koristi se metoda `generate_random_network()`:

In [ ]:
net = net_gen.generate_random_network()

Metoda kao rezultat vraća željenu mrežu odnosno instancu klase `Network`:

In [ ]:
net

koju se može vizualizirati korištenjem metode `show()`:

In [ ]:
net.show()

### Algoritam

Algoritam `Flood` dio je modula `broadcast` te ukoliko ga želimo koristiti moramo ga importirati:

In [ ]:
from pydistsim.algorithms.broadcast import Flood

i pridružiti mreži:

In [ ]:
net.algorithms = ((Flood, {"informationKey": "I"}),)

Kod pridruživanja algoritama mreži potrebno je primjetiti nekoliko stvari:

* pojedini algoritam je torka odn. `tuple` od dva elementa u kojem je prvi klasa algoritma, u ovom slučaju `Flood`, a drugi je `dict` koji se sastoji od parametara, npr. `Flood` prima kao obavezni parametar ključ pod kojim će čvorovi u svojoj memoriji spremati informaciju koja se želi podijeliti
* algoritmi se pridružuju kao elementi torke, a kako u ovom primjeru se pridružuje samo jedan algoritam potrebno je iza njega dodati zarez kako bi Python stvorio torku od jednog elementa jer primjerice `(1)` je `int`, ali `(1,)` je `tuple`

Algoritam zahtijeva postojanje barem jednog čvora koji ima informaciju, odn. inicijatora. Kako bi to bilo osigurano jednom od čvorova se ta informacija treba upisati u memoriju:

In [ ]:
some_node = net.nodes()[0]  # uzimamo prvi cvor u listi cvorova mreze
some_node.memory["I"] = (
    "Hello distributed world"  # pod kljucem 'I' upisujemo informaciju
)

### Simulacija

Nakon definicije mreže i pridruživanja algoritma potrebno je pokrenuti simulaciju. Simulaciji se pri instanciranju kao argument šalje mreža nad kojom se želi obaviti simulacija:

In [ ]:
sim = Simulation(net)

Ostaje pokrenuti simulaciju:

In [ ]:
sim.run()

Nakon što je završilo izvršavanje algoritama pogledajmo ako je informacija uspješno podijeljena tako što ćemo ispisati sadržaj memorije čvorova:

In [ ]:
for node in net.nodes():
    print node.memory['I'],

Kako bi informacija o stanju u kojem se nalazi mreža bila potpuna prati se i trenutno stanje algoritma:

In [ ]:
net.algorithmState

U ovom slučaju algoritam pod indeksom 0 je završio, uz navedeni trenutni broj koraka.

Ako se želi ponovno pokrenuti simulacija potrebno ju je resetirati:

In [ ]:
sim.reset()

čime se:

* stanje algoritama mreže postavlja na početno:

In [ ]:
net.algorithmState

* te se uz to briše i sva memorija čvorova:

In [ ]:
for node in net.nodes():
    print node.memory,

In [ ]:
some_node.memory["I"] = "Hello again"
sim.run()

## Analiza algoritma

Algoritam *Flood* je vrlo jednostavan i u cijelosti je definiran na slijedeći način:

$initiator \times \iota \longrightarrow \\{\\text{Process}(I);\; {\\bf\\text{send}}(I) \to N(x);\; {\\bf\\text{become}}\; \\text{done}\\}$

$idle \times \\text{Receiving}(I) \longrightarrow \\{\\text{Process}(I);\; {\\bf\\text{send}}(I) \to N(x) - {\\bf\\text{sender}};\; {\\bf\\text{become}}\; \\text{done}\\}$

Pogledajmo kod za modul `pydistsim.algorithms.broadcast` u kojem se nalazi algoritam `Flood`:

In [ ]:
%load '/home/ubuntu/bmo_env/src/pydistsim/pydistsim/algorithms/broadcast.py'

In [ ]:
from pydistsim.algorithm import NodeAlgorithm
from pydistsim.message import Message


class Flood(NodeAlgorithm):
    required_params = ("informationKey",)
    default_params = {"neighborsKey": "Neighbors"}

    def initializer(self):
        ini_nodes = []
        for node in self.network.nodes():
            node.memory[self.neighborsKey] = node.compositeSensor.read()["Neighbors"]
            node.status = "IDLE"
            if node.memory.has_key(self.informationKey):
                node.status = "INITIATOR"
                ini_nodes.append(node)
        for ini_node in ini_nodes:
            self.network.outbox.insert(
                0, Message(header=NodeAlgorithm.INI, destination=ini_node)
            )

    def initiator(self, node, message):
        if message.header == NodeAlgorithm.INI:
            node.send(
                Message(
                    header="Information",  # default destination: send to every neighbor
                    data=node.memory[self.informationKey],
                )
            )
            node.status = "DONE"

    def idle(self, node, message):
        if message.header == "Information":
            node.memory[self.informationKey] = message.data
            destination_nodes = list(node.memory[self.neighborsKey])
            destination_nodes.remove(message.source)  # send to every neighbor-sender
            if destination_nodes:
                node.send(
                    Message(
                        destination=destination_nodes,
                        header="Information",
                        data=message.data,
                    )
                )
        node.status = "DONE"

    def done(self, node, message):
        pass

    STATUS = {
        "INITIATOR": initiator,
        "IDLE": idle,
        "DONE": done,
    }

## Osnovne komponente

U ovom dijelu opisane su osnovne komponente PyDistSim paketa te njihov međusoban odnos.

In [ ]:
from IPython.display import Image

Image(filename="../images/pydistsim_class_diagram.png")

### Graf

PyDistSim paket se zasniva djelomično na proširenju mogućnosti paketa [NetworkX](http://networkx.lanl.gov/index.html). Dodirna točka je klasa [Graph](http://networkx.github.io/documentation/latest/reference/classes.html) koja predstavlja neusmjereni graf.

In [ ]:
from networkx import Graph

g = Graph()

`Graph` definira:

* vrhove grafa `Graph.node` -- `dict` u kojem su ključevi vrhovi odnosno bilo kakvi *immutable/hashable* python objekti, a vrijednosti njihovi (opcionalni) atributi

In [ ]:
g.add_nodes_from([1, 2, 3, 4, 5, 6])
g.node

* bridove grafa `Graph.adj` ili `Graph.edge` (isti objekt) -- `dict` u kojem su ključevi vrhovi, a vrijednost za pojedini vrh je `dict` čiji su ključevi vrhovi susjedi zadanog vrha, a vrijednosti su (opcionalni) atributi brida

In [ ]:
g.add_edges_from([(1,2),(1,3),(2,3),(3,4),(4,5),(5,6),(1,6),(5,2)])
print g.edge is g.adj
g.edge

* razne metode za manipulaciju strukturom grafa

In [ ]:
g.add_star(g.nodes())
g.edge

* [funkcije za iscrtavanje](http://networkx.github.io/documentation/latest/reference/drawing.html)

In [ ]:
from networkx import draw_networkx

draw_networkx(g)

* [algoritme](http://networkx.github.io/documentation/latest/reference/algorithms.html), kao npr. [shortest_path](http://networkx.github.io/documentation/latest/reference/generated/networkx.algorithms.shortest_paths.generic.shortest_path.html)

In [ ]:
from networkx.algorithms import shortest_path

shortest_path(g, 2, 6)

Dodatno NetworkX paket sadrži i:

* druge tipove grafova (usmjereni, multigraf...)

In [ ]:
from networkx import DiGraph

dg = DiGraph([(1, 2), (2, 1), (1, 3), (3, 2)])
draw_networkx(dg)

* [generatore grafova](http://networkx.github.io/documentation/latest/reference/generators.html), kao npr. [hypercube_graph](http://networkx.github.io/documentation/latest/reference/generated/networkx.generators.classic.hypercube_graph.html#networkx.generators.classic.hypercube_graph)

In [ ]:
from networkx.generators.classic import hypercube_graph

hcg = hypercube_graph(3)
draw_networkx(hcg)

 * [konvertere](http://networkx.github.io/documentation/latest/reference/convert.html) itd.

In [ ]:
mat = numpy.random.random((6,6))>0.5
print mat
from networkx.convert import from_numpy_matrix
g = from_numpy_matrix(mat)
draw_networkx(g)

### Mreža

`Network` je osnovna klasa PyDistSim simulatora. Njene instance sadrže sve informacije potrebne kako bi se za pojedinu mrežu pokrenula ili nastavila simulacija njoj pripadajućih algoritama. Kao što je već naglašeno klasa `Network` je podklasa (podrazred) klase Graph iz paketa NetworkX.

Uz sve značajke grafa PyDistSim u klasi `Network` proširuje mogućnosti sa slijedećim atributima i funkcionalnostima:

* `Network.environment` - trenutno je omgućeno samo 2D okruženje, kroz instancu klase `Environment2D`

In [ ]:
from pydistsim.environment import Environment2D

o_env = Environment2D(path="../images/o_shape.png")
c_env = Environment2D(path="../images/c_shape.png")
o_net = Network(environment=o_env)
c_net = Network(environment=c_env)
for _ in range(100):
    o_net.add_node()
    c_net.add_node()
o_net.show()
c_net.show()

* `Network.pos` - lokacija čvorova u okruženju

In [ ]:
net.pos

* `Network.algorithms` - torka algoritama: svi čvorovi izvršavaju iste algoritme, pa je mreža adekvatno mjesto za definiciju

In [ ]:
net.algorithms

* `Network.channelType` - model komunikacijskog kanala

In [ ]:
net.channelType

In [ ]:
# %load '/home/ubuntu/bmo_env/src/pydistsim/pydistsim/channeltype.py'
from pydistsim.channeltype import Udg, SquareDisc

udg = Udg(net.environment)
scd = SquareDisc(net.environment)
udg_cr = []
scd_cr = []
distances = range(1, 201, 10)
net = Network()
node1 = net.add_node(pos=[1, 1])
for i, distance in enumerate(distances):
    node2 = net.add_node(pos=[1.1, distance])
    udg_cr.append(mean([udg.in_comm_range(net, node1, node2) for j in range(10)]))
    scd_cr.append(mean([scd.in_comm_range(net, node1, node2) for j in range(10)]))
plt.ylim(0, 1.1)
plot(distances, udg_cr)
plot(distances, scd_cr)

* model komunikacije kroz `Network.outbox` i `Network.communicate()`

Mreža je središnji objekt u PyDistSim paketu i ona sadrži sve potrebne informacije kako bi se rekreirao eksperiment ili sačuvali podaci za kasniju analizu. Shodno tome implementirane su metode za njeno spremanje i čitanje s diska:

In [ ]:
write_npickle(net, "mreza1.tar.gz")
net_from_file = read_npickle("mreza1.tar.gz")
net_from_file

In [ ]:
!ls -algo mr*

### Algoritmi

PyDistSim podržava dva tipa algoritama raspodijeljeni i centralizirani:

* *Centralizirani* algoritmi se implementiraju kao podklase od `NetworkAlgorithm` i funkcioniraju po prinicipu izravnog upisivanja podataka u memoriju čvorova.
* *Raspodijeljeni* algoritmi se implementiraju kao podklase od `NodeAlgorithm`. Raspodijeljeni algoritmi mogu koristiti samo informacije spremljene u samoj memoriji čvora te one dobivene očitanjem osjetila koje pojeduje čvor.

### Čvorovi

Čvorovi su instance klase `Node` koja između ostalog implementira slijedeće atribute i metode:

* `memory` - `dict` u kojem čvor čuva cjelokupno znanje koje može prikupiti na dva načina:
 * očitanje osjetila - npr. polje `Neighbors` u algoritmu `Flood`
 * informacije u primljenim porukama - npr. polje `I` u algoritmu `Flood`

In [ ]:
some_node.memory["Neighbors"]

In [ ]:
some_node.memory["I"]

* `status` - registar u memoriji s posebnim značenjem za raspodijeljene algoritme implementiran je kao poseban atribut

In [ ]:
some_node.status

* `commRange` - komunikacijski domet čvora: u kombinaciji s pozicijom, okruženjem i modelom kominikacijskog kanala određuje susjede pojedinog čvora

In [ ]:
some_node.commRange

* `outbox` i `_inbox` - liste u kojima su spremeljene poruke (instance klase `Message`) spremene za slanje odnosno primljene poruke
* `send()` - metoda za slanje poruka drugim čvorovima
* `compositeSensor` - predstavlja torku osjetila koji su instalirani na čvoru te implementira metodu `read()` koja vraća `dict` s ključevima koji odgovaraju pojedinom osjetilu, a vrijednosti su njihova trenutna očitanja

In [ ]:
print some_node.compositeSensor
print some_node.compositeSensor.sensors
some_node.compositeSensor.read()

### Osjetila

Osjetila predstavljaju sučelje čvora prema okolini. Impelentirani su kao podklase apstraktne klase `Sensor`. Implementiraju metodu `read()` koja u ovisnosti o osjetilu može svoje podatke dobiti uvidom u mrežu odnosno okruženje.

In [ ]:
from pydistsim.sensor import TruePosSensor

new_net = Network()
node = new_net.add_node(pos=[100, 100])
node.compositeSensor = (TruePosSensor, "DistSensor")
new_net.add_node(pos=[170, 170])
node.compositeSensor.read()

Zgodno je primjetiti kako je prvo osjetilo pridruženo kao klasa, a drugo samo imenom klase. I jedna i druga opcija su moguće.

Pojedino osjetilo može implementirati atribut `probabilityFunction` koji mu omogućava 'zašumljivanje' dobivenih podataka kako bi očitanje bilo bliže stvarnom očitanju koje je u većini slučajeva podložno manjim ili većim mjernim nesigurnostima.

In [ ]:
dist_sensor = node.compositeSensor.sensors[0]
dist_sensor.probabilityFunction.pf

In [ ]:
distances = []
for i in range(1000):
    distances.append(node.compositeSensor.read()["Dist"].values()[0])
h = hist(distances, bins=30)

Osjetila koje određeni čvor posjeduje mogu promijeniti njegovu ulogu unutar pojeding algoritma. Primjerice kod algoritma lokalizacije čvor s `TruePos` osjetilom predstavlja sidro.

Osjetila služe i kako bi se implementirala određene pretpostavke, nazvane još i ograničenja algoritma. Primjerice, ukoliko algoritam zahtijeva za svaki čvor poznavanje svojih susjeda tada se u svakom čvoru postavlja osjetilo `NeighborsSensor` čijim očitanjem čvor u svakom trenutku može 'očitati' svoje susjede.

### Aktuatori

U planu, uključuje mobilne čvorove, kinematički model, dinamičke grafove.

### Simulacija

Klasa `Simulation` se brine za izvršavanje algoritma u pripadajućoj mreži `Simulation.network`.

Između ostalog, brine se i za osvježavanje grafičkog sučelja simulatora.

<!--- Definirana je kao dretva (*thread*) odnosno podklasa od klase `QtCore.QThread` paketa Pyside.
-->

### Postavke

Kako bi rad u interaktivnoj konzoli, grafičkom sučelji ili u automatiziranim eksperimentima bio olakšan potrebno je preddefinirati određene vrijednosti. Takve vrijednosti definiraju se u postavkama `pydistsim.conf.settings`.

Trenutne globalne postavke:

In [ ]:
%load '/home/ubuntu/bmo_env/src/pydistsim/pydistsim/conf/global_settings.py'

In [ ]:
"""Default pydistsim settings.

Override these with settings in the module pointed-to by the
PYDISTSIM_SETTINGS_MODULE environment variable or by using
settings.configure(**settings) or settings.load('path.to.settings')

"""

import scipy.stats
from numpy import pi

###########
# NETWORK #
###########
ENVIRONMENT = "Environment2D"
ENVIRONMENT2D_SHAPE = (600, 600)


ALGORITHMS = ()
# ALGORITHMS = ((ReadSensors,
#               {'sensorReadingsKey':'sensorReadings'}),
#              )

CHANNEL_TYPE = "Udg"


##########
#  NODE  #
##########
SENSORS = "NeighborsSensor"
# SENSORS = ('AoASensor','DistSensor')
ACTUATORS = ()
COMM_RANGE = 100

AOA_PF_PARAMS = {"pf": scipy.stats.norm, "scale": 10 * pi / 180}  # in radians
DIST_PF_PARAMS = {"pf": scipy.stats.norm, "scale": 10}

Postavke se mogu promijeniti odnosno prepisati (*override*-ati) na nekoliko načina:

* pisanjem novog modula settings.py na kojeg se referencira *environment* varijabla PYDISTSIM_SETTINGS_MODULE u npr. 'paket.podpaket.settings'
* prije prvog korištenja postavki one se mogu promijeniti korištenjem settings.configure(**settings) ili
* korištenjem settings.load('paket.podpaket.settings') u bilo kojem trenutku

## Teme za domaću zadaću:

1. **Pervan**: Shout, MultiShout
2. **Vidović**: DF_Traversal: centralizirani i raspodijeljeni
3. **Kegalj**: DF*
4. **Grbac**: SmartTraversal: Shout+ i DF_Traversal
5. **Frantal**: TD_Cast: Flood with reply i Convergecast
6. **Barak**: FullSaturation primjenjen na Minimum finding i Average
7. **Tuhtan**: FullSaturation primjenjen na Eccentricities i Center finding


Napomene:

* Projekt voditi na githubu ili bitbucketu (podijeliti prisup repozitoriju, ako je privatni)
* *checkpoint* idući tjedan, prezentacije za dva tjedna,
* Pripremiti IPython notebookove i/ili simulacije u GUI-u.

Za version control na githubu i bitbucketu se koriste slijedeći sustavi:

### Mercurial

http://hginit.com/

### Git

http://nakedstartup.com/2010/04/simple-daily-git-workflow

### Distribucija paketa (opcija)

http://docs.python.org/2/distutils/index.html#distutils-index

### Windows

U datoteci ``C:\Users\<user>\.pypirc``:

    [distutils]
    index-servers =
        pypi

    [pypi]
    repository: http://pypi.python.org/pypi
    username: <username>
    password: <password>

    [server-login]
    repository: http://pypi.python.org/pypi
    username: <username>
    password: <password>

Za distribuciju pokrenuti slijedeće naredbe:

    > setx HOME C:\Users\<user>
    > python setup.py sdist upload register

## GUI

Grafičko sučelje simulatora pokreće se:

1. u *standalone* verziji korištenjem instaliranog programa `pydistsim-simgui`
2. iz interaktivne konzole sa `%run pydistsim/gui/simulationgui.py`

Prednost druge metode je u tome što se paralelno sa sučeljem u konzoli može pristupiti svim objektima koristeći objekt `simgui` npr. `simgui.network` predstavlja mrežu koja je trenutno u sučelju.